In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
from dataclasses import dataclass
rng = np.random.default_rng()

# Shots Table

In [3]:
df = pd.read_excel("Synthetic Data.xlsx", sheet_name=1)
df.head()

,type,x,y,isgoal,appearance
0,Jump,0,7,False,0
1,Standing,25,25,True,0
2,Dive,18,7,True,0
3,Wing,11,7,True,0
4,Lob,9,1,False,2


In [4]:
def generate_synthetic(df, n_rows, schema):
    synthetic = pd.DataFrame(index=range(n_rows))
    start_time=pd.Timestamp("2026-01-01 00:00")

    for col in schema:
        synthetic[col.name] = rng.integers(
            size=n_rows,
            **col.params
        )
    
    for col in df.select_dtypes(exclude=np.number):
        freq = df[col].value_counts(normalize=True)
        synthetic[col] = rng.choice(
            freq.index,
            size=n_rows,
            p=freq.values
        )

    synthetic["timestamp"] = start_time + pd.to_timedelta(
        rng.uniform(0, 3600, size=n_rows),
        unit="s"
    )
    
    return synthetic

In [5]:
@dataclass
class ColumnDescription:
    name: str
    params: dict

schema = [
    ColumnDescription("appearance", {"low": 0, "high": 25}),
    ColumnDescription("x", {"low": 0, "high": 27}),
    ColumnDescription("y", {"low": 0, "high": 27}),
]


In [6]:
synth = generate_synthetic(df, n_rows=500, schema=schema)
synth["time"] = synth["timestamp"].dt.time
#synth = synth.drop(["timestamp"], axis=1)
synth = synth.sort_values(by=["appearance", "time"]).reset_index(drop=True)
synth.head(15)

,appearance,x,y,type,isgoal,timestamp,time
0,0,8,15,Spin,True,2026-01-01 00:04:14.900087731,00:04:14.900087
1,0,14,1,Spin,True,2026-01-01 00:05:28.141441708,00:05:28.141441
2,0,22,7,Lob,False,2026-01-01 00:09:52.878587494,00:09:52.878587
3,0,21,14,Bounce,False,2026-01-01 00:09:57.580197069,00:09:57.580197
4,0,11,9,Bounce,True,2026-01-01 00:11:11.134252590,00:11:11.134252
5,0,13,3,Wing,False,2026-01-01 00:11:29.468752940,00:11:29.468752
6,0,13,2,Wing,True,2026-01-01 00:16:57.472112620,00:16:57.472112
7,0,8,21,Spin,False,2026-01-01 00:20:53.724281959,00:20:53.724281
8,0,24,13,Standing,False,2026-01-01 00:24:00.321531387,00:24:00.321531
9,0,8,26,Hip,False,2026-01-01 00:24:20.176925194,00:24:20.176925


In [7]:
synth["appearance"].value_counts().sort_values(ascending=False)

appearance
4     30
1     25
23    24
19    23
21    23
9     22
0     22
14    22
5     21
11    21
12    21
8     20
10    20
13    19
15    19
7     19
6     19
16    18
3     17
20    17
22    16
24    16
18    16
2     15
17    15
Name: count, dtype: int64

# Appearances Table

In [8]:
app_df = synth.groupby("appearance")["isgoal"].sum().reset_index(name="total_goals")
app_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   appearance   25 non-null     int64
 1   total_goals  25 non-null     int64
dtypes: int64(2)
memory usage: 532.0 bytes


In [9]:
n_players = 4


player_ids = np.arange(0, n_players)
player_probs = np.array([0.5, 0.25, 0.15, 0.1], dtype=float)
player_probs = player_probs / player_probs.sum()
app_df["player_id"] = rng.choice(player_ids, size=len(app_df), p=player_probs)


# atribuir posição a cada jogador (0-1 GR, 2-3 avançados)
player_positions = {
    0: "GK",
    1: "GK",
    2: "ST",
    3: "ST",
}
app_df["position"] = app_df["player_id"].map(player_positions)


app_df["match_id"] = rng.integers(0, 23, size=len(app_df))

In [10]:
app_df["player_id"].value_counts()

player_id
0    14
1     5
2     4
3     2
Name: count, dtype: int64

In [11]:
app_df.head(10)

,appearance,total_goals,player_id,position,match_id
0,0,8,0,GK,0
1,1,8,2,ST,22
2,2,6,1,GK,8
3,3,7,0,GK,8
4,4,8,1,GK,1
5,5,9,2,ST,3
6,6,14,0,GK,7
7,7,7,3,ST,1
8,8,7,0,GK,20
9,9,12,0,GK,3


# Players Table

In [12]:
player_df = app_df.groupby(["player_id", "position"])["total_goals"].sum().reset_index(name="total_goals")


# gerar stats físicas e mentais específicas por posição


gk_template = {
    "reflexes": (70, 95),
    "handling": (65, 90),
    "aerial_command": (70, 95),
    "one_v_one": (70, 95),
    "communication": (60, 90),
}


st_template = {
    "finishing": (70, 95),
    "off_ball": (65, 90),
    "pace": (70, 95),
    "strength": (60, 90),
    "pressing": (60, 90),
}


for idx, row in player_df.iterrows():
    pos = row["position"]
    if pos == "GK":
        template = gk_template
    else:
        template = st_template
    for stat_name, (low, high) in template.items():
        player_df.loc[idx, stat_name] = rng.integers(low, high)


player_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   player_id       4 non-null      int64  
 1   position        4 non-null      object 
 2   total_goals     4 non-null      int64  
 3   reflexes        2 non-null      float64
 4   handling        2 non-null      float64
 5   aerial_command  2 non-null      float64
 6   one_v_one       2 non-null      float64
 7   communication   2 non-null      float64
 8   finishing       2 non-null      float64
 9   off_ball        2 non-null      float64
 10  pace            2 non-null      float64
 11  strength        2 non-null      float64
 12  pressing        2 non-null      float64
dtypes: float64(10), int64(2), object(1)
memory usage: 548.0+ bytes


In [13]:
player_df.head()

,player_id,position,total_goals,reflexes,handling,aerial_command,one_v_one,communication,finishing,off_ball,pace,strength,pressing
0,0,GK,126,71.0,72.0,80.0,88.0,76.0,NaN,NaN,NaN,NaN,NaN
1,1,GK,39,77.0,78.0,75.0,90.0,68.0,NaN,NaN,NaN,NaN,NaN
2,2,ST,32,NaN,NaN,NaN,NaN,NaN,70.0,83.0,85.0,66.0,84.0
3,3,ST,16,NaN,NaN,NaN,NaN,NaN,78.0,65.0,87.0,87.0,88.0


# Matches Table

In [14]:
match_df = app_df.groupby("match_id")["total_goals"].sum().reset_index(name="conceded")
match_df.head(10)

,match_id,conceded
0,0,33
1,1,15
2,2,8
3,3,29
4,5,10
5,6,9
6,7,21
7,8,13
8,9,13
9,12,8


In [15]:
with pd.ExcelWriter('output.xlsx') as writer:
    synth.to_excel(writer, sheet_name='Shots', index=False)
    app_df.to_excel(writer, sheet_name='Appearances', index=False)
    player_df.to_excel(writer, sheet_name='Players', index=False)
    match_df.to_excel(writer, sheet_name='Matches', index=False)